# Training a Chatbot in Hindi using iNLTK
iNLTK aims to provide out of the box support for various NLP tasks that an application developer might need in Indic Languages

In [ ]:
pip install inltk

In [ ]:
pip install tflearn

In [8]:
from inltk.inltk import setup
from inltk.inltk import tokenize

In [ ]:
setup('hi')

In [11]:
# Importing all the necessary modules
import numpy
import tflearn
import tensorflow
import random
import json
import pickle

Instructions for updating:
non-resource variables are not supported in the long term


In [19]:
# Load the data
with open("firstaid_hindi.json") as file:
    data = json.load(file)

In [21]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
	for pattern in intent["patterns"]:
		wrds = tokenize(pattern, 'hi')
		words.extend(wrds)
		docs_x.append(wrds)
		docs_y.append(intent["tag"])

	if intent["tag"] not in labels:
		labels.append(intent["tag"])

words = [w for w in words if w != "?"]

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
	bag = []

	wrds = [(w) for w in doc]

	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
	output_row = out_empty[:]
	output_row[labels.index(docs_y[x])] = 1

	training.append(bag)
	output.append(output_row)

training = numpy.array(training)
output = numpy.array(output)

In [ ]:
tensorflow.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

In [23]:
try:
	model.load("firstaid_hindi_model.tflearn")
except:
	model = tflearn.DNN(net)
	model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
	model.save("firstaid_hindi_model.tflearn")

Training Step: 1999  | time: 0.003s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 1.0000 -- iter: 08/16
Training Step: 2000  | time: 0.014s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 1.0000 -- iter: 16/16
--
INFO:tensorflow:/content/firstaid_hindi_model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [24]:
def bag_of_words(s,words):
	bag = [0 for _ in range(len(words))]


	s_words = tokenize(s, 'hi')

	for se in s_words:
		for i, w in enumerate(words):
			if w == se:
				bag[i] = 1

	return numpy.array(bag)

In [25]:
def chat():
	print("Start Talking with the bot(type quit to stop!")
	while True:
		inp = input("You: ")
		if inp.lower() == "quit":
			break

		results = model.predict([bag_of_words(inp,words)])[0]
		results_index = numpy.argmax(results)
		tag = labels[results_index]

		if results[results_index] > 0.5:
			for tg in data["intents"]:
				if tg['tag'] == tag:
					responses = tg['responses']
			print(random.choice(responses))
			print("\n")

		else:
			print("मुझे समझ नहीं आया, कृपया फिर से टाईप करें।")

In [27]:
chat() 

Start Talking with the bot(type quit to stop!
You: हड्डी टूटना
1) अस्थिभंग (fracture) वाले स्थान को पटरियों तथा अन्य उपायों से अचल बनाए बिना रोगी को स्थानांतरित न करें। 2) चोट के स्थान से यदि रक्तस्त्राव हो रहा हो तो प्रथमतः उसका उपचार करें। 3) बड़ी चौकसी के साथ बिना बल लगाए, अंग को यथासाध्य अपने स्वभाविक स्थान पर बैठा दें। 4) चपतियों (splints), पट्टियों (bandages) और लटकानेवाली पट्टियों, अर्थात् झोलों, के प्रयोग से भग्न अस्थिवाले भाग को यथासंभव स्वाभाविक स्थान पर बनाए रखने की चेष्टा करें। 5) जब संशय हो कि हड्डी टूटी है या नहीं, तब भी उपचार उसी भाँति करें जैसा हड्डी टूटने पर होना चाहिए।


You: अचेतनावस्था
बेहोशी पैदा करनेवाले कारणों से घायल को दूर कर देना तथा अचेतनावस्था के उपचार के साधारण नियमों को यथासंभव काम में लाना चाहिए।


You: quit
